In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

%matplotlib inline

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Теперь-реализуем-собственный-алгоритм." data-toc-modified-id="Теперь-реализуем-собственный-алгоритм.-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Теперь реализуем собственный алгоритм.</a></span></li>

# Защита персональных данных клиентов

Нам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработаем такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

## Загрузка данных

Загрузим данные и посмотрим, имеются ли пропуски или данные столбцов приведены не к тому типу.

In [2]:
try:
    data = pd.read_csv('/datasets/insurance.csv')

except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')

data.info()
data.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Повезло, данные в полном порядке и пропусков нет!

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

Проверим, измениться ли качество линейной регрессии, при умножении признаков на обратимую матрицу.

###### Шаг 1. Введем матрицу Z.
Пусть Z - наша новая матрица $$Z = XP$$
<br></br>
###### Шаг 2. Выведем обновленную формулу для предсказания
$$a1 = Zw = Z (Z^T Z)^{-1} Z^T y $$
###### Шаг 3. Раскроем нашу матрицу Z
$$a1 = XP ((XP)^{T} XP))^{-1} (XP)^T y$$
###### Шаг 4. Преобразовываем выражения $$(XP)^T = P^T X^T$$и упрощаем
$$a1 = XP(P^TX^TXP)^{-1} P^TX^T y$$
$$a1 = XP (P)^{-1} (X^TX)^{-1} (P^T)^{-1} P^T X^T y$$
сокращаем обратные друг другу выражения
$$a1 = X(X^T X)^{-1} X^T y  $$
###### Шаг 4. Сопоставляем
$$a1 = a$$
так как
$$(X^T X)^{-1} X^T y = w$$
по условию и $$Xw = X(X^T X)^{-1} X^T y$$
Формула обучения не изменилась - качество линейной регрессии не изменится.


## Алгоритм преобразования

**Алгоритм**

Алгоритм шифрования, суть которого умножить матрицу признаков на обратимую матрицу.
<br></br>
- Генерируем и фиксируем случайную квадратную матрицу в качестве ключа шифрования размером, равному количеству признаков, не учитывая целевой.
<br></br>
- Скалярно умножаем матрицу признаков на нашу обратимую квадратную матрицу, полученную в первом шаге.
<br></br>
- Используем полученное произведение в качестве features для разбития данных и обучения модели.

**Обоснование**

Мы производим умножение каждого набора признаков из features на одинаковые наборы векторов (столбец - вектор) из квадратной обратимой матрицы и записываем их в новый вектор - строку. Соответственно итоговые веса каждого такого вектора будут равны весам набора признаков.

В Пункте 2 мы уже доказали, почему качество не снизиться - выражения тождественны.

## Проверка алгоритма

Обозначим целевой и нецелевые признаки. Как нам известно, целевым считается признак *Страховые выплаты*.

In [3]:
features = data.drop('Страховые выплаты', axis=1)
target = data['Страховые выплаты']

Разобьем данные на обучающую и тестовую выборки в соотношении 3:1.

In [4]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2,
                                                                            random_state=12345)

Обучим модель и подсчитаем метрику *r2 score*.


In [5]:
from sklearn.metrics import r2_score

model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('R2 score = ', np.round(r2_score(y_true=target_test, y_pred=predictions), 5))

R2 score =  0.41177


## Теперь реализуем собственный алгоритм.

Создадим квадратную обратимую матрицу методом np.random.normal, при использовании этого метода вероятность получить необратимую матрицу близка к нулю.
В size передадим размер признаков - 1, так как один из них целевой.

In [6]:
square_matrix = np.random.normal(size=(data.shape[1] - 1, data.shape[1] - 1))
square_matrix

array([[-0.12954955,  0.59662364,  0.1716113 , -0.9852623 ],
       [ 0.26223839,  1.0747992 ,  0.28864125, -0.12068828],
       [ 1.45375467, -0.25287972, -0.05163723,  0.78913095],
       [-0.87614381,  1.52586754, -0.34181388, -0.34817173]])

Здесь мы проверим, что получаем единичную матрицу при перемножении матрицы на её обратную.

In [7]:
inverse_square_matrix = np.linalg.inv(square_matrix)
square_matrix @ inverse_square_matrix

array([[ 1.00000000e+00, -1.79310772e-17,  3.16241951e-17,
         3.14255638e-17],
       [ 3.67417878e-17,  1.00000000e+00,  1.31002022e-16,
         1.21128125e-16],
       [-3.02125209e-16,  1.02331483e-17,  1.00000000e+00,
        -2.95836171e-18],
       [ 7.77510070e-17,  7.98418285e-17,  2.09903577e-17,
         1.00000000e+00]])

Создадим наши зашифрованные признаки путем скалярного умножения на обратимую матрицу.

In [8]:
new_features = features @ square_matrix

Разбиваем данные.

In [9]:
new_features_train, new_features_test, new_target_train, new_target_test = train_test_split(new_features, target,
                                                                                            test_size=0.2,
                                                                                            random_state=12345)

И наконец считаем метрику.

In [10]:
new_model = LinearRegression()
new_model.fit(new_features_train, new_target_train)
new_predictions = new_model.predict(new_features_test)
print('R2 score = ', np.round(r2_score(y_true=new_target_test, y_pred=new_predictions), 5))

R2 score =  0.41177


Метрика Линейной Регрессии по умолчании и метрика с учетом нашим зашифрованных данных равны! То бишь наш алгоритм никак не повлиял на качество.

В данном проекте был проведен просмотр и анализ данных. Повезло - они были в полном порядке без пропусков и с правильным приведением типов. Далее был исследован вопрос, заключающийся в умножении признаков на обратимую матрицу, изменится ли качество линейной регрессии? Мы получили отрицательный ответ на вопрос доказав, что при обычном и в нашем вопросе формулы обучения тождественны, что говорит о том, что качество линейной регрессии не изменится, а вот данные как раз таки зашифруются. Далее использовали данный алгоритм и применили его на практике с нашими данными: проверили на Обычной Линейной Регрессии и с учетом умножения признаков на обратимую матрицу и как оказалось - метрика R2 Score - одинакова и составляет 0.41177 (без учета подбора гипперпараметров).